In [1]:
import pandas as pd
import requests
import numpy as np
from bs4 import BeautifulSoup 

In [2]:
base_url = 'https://www.dsebd.org/companylistbyindustry.php?industryno='

In [3]:
page = requests.get('https://www.dsebd.org/companylistbyindustry.php?industryno=12', verify=False).content

c:\users\rapples\desktop\quantitative\portfolio-analysis\sy-portfolio\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


In [4]:
soup = BeautifulSoup(page)

body > table:nth-child(11) > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table:nth-child(4) > tbody > tr:nth-child(1) > td > table > tbody > tr > td > table


In [5]:
table = soup.select("body > table:nth-child(10) > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td > table > tr > td")[0]
table = table.find_all('table')[2]
table = table.find_all('table')[0]
table = pd.read_html(table.prettify())[0]
table = table[0].str.split(' ', expand=True)[0]
table.values

array(['1JANATAMF', '1STPRIMFMF', 'ABB1STMF', 'AIBL1STIMF', 'ATCSLGF',
       'CAPMBDBLMF', 'CAPMIBBLMF', 'DBH1STMF', 'EBL1STMF', 'EBLNRBMF',
       'EXIM1STMF', 'FBFIF', 'GRAMEENS2', 'GREENDELMF', 'ICB3RDNRB',
       'ICBAGRANI1', 'ICBAMCL2ND', 'ICBEPMF1S1', 'ICBSONALI1',
       'IFIC1STMF', 'IFILISLMF1', 'LRGLOBMF1', 'MBL1STMF', 'NCCBLMF1',
       'NLI1STMF', 'PF1STMF', 'PHPMF1', 'POPULAR1MF', 'PRIME1ICBA',
       'RELIANCE1', 'SEBL1STMF', 'SEMLFBSLGF', 'SEMLIBBLSF', 'SEMLLECMF',
       'TRUSTB1MF', 'VAMLBDMF1', 'VAMLRBBF'], dtype=object)

In [6]:
industry_page = BeautifulSoup(requests.get('https://www.dsebd.org/by_industrylisting1.php', verify=False).content, 'lxml')

c:\users\rapples\desktop\quantitative\portfolio-analysis\sy-portfolio\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)


body > table:nth-child(11) > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td > table > tbody > tr > td > table > tbody > tr > td:nth-child(1) > table

In [7]:
industries = industry_page.select('body > table:nth-child(10) > tbody > tr > td:nth-child(2) > table > tbody > tr > td:nth-child(1) > table')[0]
industries = industries.find_all('table')[1]
industries_df = pd.read_html(industries.prettify())[0]
industries_df =  industries_df.rename(industries_df.iloc[0], axis=1).drop('Detail', axis=1).drop('#', axis=1)[1:]
industries_df

,Name of the Industry,Quantity
1,Bank,30
2,Cement,7
3,Ceramics Sector,5
4,Corporate Bond,1
5,Debenture,8
6,Engineering,39
7,Financial Institutions,23
8,Food & Allied,17
9,Fuel & Power,19
10,Insurance,47


In [8]:
industries_links = industries.find_all(href=True)
df_links = pd.DataFrame()
for link in industries_links:
    num = link['href'].split('=')[1]
    text = str(link.text)
    df_links = df_links.append({'Name': text, 'No': num}, ignore_index= True)
df_links = df_links[df_links['Name'] != df_links['Name'].iloc[1]]
industry_names = df_links['Name'][:-1]
industry_names

0                            Bank
2                          Cement
4                 Ceramics Sector
6                  Corporate Bond
8                       Debenture
10                    Engineering
12         Financial Institutions
14                  Food & Allied
16                   Fuel & Power
18                      Insurance
20                      IT Sector
22                           Jute
24                  Miscellaneous
26                   Mutual Funds
28               Paper & Printing
30    Pharmaceuticals & Chemicals
32         Services & Real Estate
34             Tannery Industries
36              Telecommunication
38                        Textile
40               Travel & Leisure
Name: Name, dtype: object

In [9]:
industry_companies = pd.DataFrame()
for industry in industry_names.values:
    nol = df_links[df_links['Name']==industry]['No'].values
    url = base_url + nol
    page = BeautifulSoup(requests.get(url[0], verify=False).content, 'lxml').select("body > table:nth-child(10) > tbody > tr > td:nth-child(2) > table > tbody > tr:nth-child(2) > td > table > tr > td")[0]
    table = page.find_all('table')[2]
    table = table.find_all('table')[0]
    table = pd.read_html(table.prettify())[0]
    table = table[0].str.split(' ', expand=True)[0]
    df_tmp = pd.DataFrame({industry:table.values})
    industry_companies = pd.concat([industry_companies,df_tmp], axis=1)
    

c:\users\rapples\desktop\quantitative\portfolio-analysis\sy-portfolio\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
c:\users\rapples\desktop\quantitative\portfolio-analysis\sy-portfolio\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#ssl-warnings
  InsecureRequestWarning)
c:\users\rapples\desktop\quantitative\portfolio-analysis\sy-portfolio\venv\lib\site-packages\urllib3\connectionpool.py:851: InsecureRequestWarning: Unverified HTTPS request is being made. Adding certificate verification is strongly advised. See: https://urllib3.readthedocs.io/en/latest/advanced-usage.html#s

In [10]:
industry_companies.to_csv('./companis-by-industry.csv')

In [11]:
eng_companies = industry_companies['Pharmaceuticals & Chemicals'].dropna().values
eng_companies

array(['ACI', 'ACIFORMULA', 'ACMELAB', 'ACTIVEFINE', 'ADVENT', 'AFCAGRO',
       'AMBEEPHA', 'BEACONPHAR', 'BXPHARMA', 'BXSYNTH', 'CENTRALPHL',
       'FARCHEM', 'GHCL', 'GLAXOSMITH', 'IBNSINA', 'IBP', 'IMAMBUTTON',
       'JMISMDL', 'KEYACOSMET', 'KOHINOOR', 'LIBRAINFU', 'MARICO',
       'ORIONINFU', 'ORIONPHARM', 'PHARMAID', 'RECKITTBEN', 'RENATA',
       'SALVOCHEM', 'SILCOPHL', 'SILVAPHL', 'SQURPHARMA', 'WATACHEM'],
      dtype=object)